In [1]:
from ortools.sat.python import cp_model
import time
import pandas
import numpy as np
import itertools

Initialise model

In [2]:
model = cp_model.CpModel()

Route \<generated from route scheduling\>

In [3]:
np.random.seed(seed=1)

points_start = ['S1', 'S2', 'S3']
points_holding_1 = ['Hb1', 'Hb2']
points_holding_2 = ['Ha1', 'Ha2']
points_holding_dict = dict()
for i, j in zip(points_holding_1, points_holding_2):
    points_holding_dict.update({i: j, j:i})
points_destination = ['D1', 'D2', 'D3']
points = points_start + points_holding_1 + points_holding_2 + points_destination

routes = pandas.DataFrame(999, index=points, columns=points)
for s in points_start:
    for h in points_holding_1:
        routes.loc[s, h] = np.random.randint(5, 10)
for h in points_holding_2:
    for d in points_destination:
        routes.loc[h, d] = np.random.randint(7, 13)
for i in range(routes.shape[0]):
    for j in range(routes.shape[1]):
        routes.iloc[j, i] = routes.iloc[i, j]
routes

,S1,S2,S3,Hb1,Hb2,Ha1,Ha2,D1,D2,D3
S1,999,999,999,8,9,999,999,999,999,999
S2,999,999,999,5,6,999,999,999,999,999
S3,999,999,999,8,5,999,999,999,999,999
Hb1,8,5,8,999,999,999,999,999,999,999
Hb2,9,6,5,999,999,999,999,999,999,999
Ha1,999,999,999,999,999,999,999,7,8,11
Ha2,999,999,999,999,999,999,999,12,11,8
D1,999,999,999,999,999,7,12,999,999,999
D2,999,999,999,999,999,8,11,999,999,999
D3,999,999,999,999,999,11,8,999,999,999


Variables

In [4]:
num_personnel = 5
vehicles_alpha = ['A1']
vehicles_beta = ['B1', 'B2']
points_launch = ['S1', 'S3']   ## these nodes will not be visited again

alpha_p, beta_p = {}, {}
for n in range(num_personnel):
    for a in vehicles_alpha:
        for h in points_holding_2:
            for d in points_destination:
                alpha_p[(n, a, h, d)] = model.NewBoolVar(f'x_{n}{a}{h}{d}')
for n in range(num_personnel):
    for b in vehicles_beta:
        for s in points_start:
            for h in points_holding_1:
                beta_p[(n, b, s, h)] = model.NewBoolVar(f'y_{n}{b}{s}{h}')

alpha_v, beta_v = {}, {}
for v in vehicles_alpha:
    for h in points_holding_2:
        for d in points_destination:
            alpha_v[(v, h, d)] = model.NewBoolVar(f'p_{v}{h}{d}')
            alpha_v[(v, d, h)] = model.NewBoolVar(f'p_{v}{d}{h}')
for v in vehicles_beta:
    for s in points_start:
        for h in points_holding_1:
            beta_v[(v, s, h)] = model.NewBoolVar(f'q_{v}{s}{h}')
            beta_v[(v, h, s)] = model.NewBoolVar(f'q_{v}{h}{s}')

node_p = {}
for n in range(num_personnel):
    for i in points_start + points_holding_1 + points_holding_2 + points_destination:
        node_p[(n, i)] = model.NewBoolVar(f'z_{n}{i}')

node_v = {}
for v in vehicles_alpha:
    for i in points_holding_2 + points_destination:
        node_v[(v, i)] = model.NewBoolVar(f'w_{v}{i}')
for v in vehicles_beta:
    for i in points_start + points_holding_1:
        node_v[(v, i)] = model.NewBoolVar(f'r_{v}{i}')

departure, arrival = {}, {}
for v in vehicles_alpha:
    for i in points_holding_2 + points_destination:
        departure[(v, i)] = model.NewIntVar(0, 10000, f'd_{v}{i}')
        arrival[(v, i)] = model.NewIntVar(0, 10000, f'a_{v}{i}')
for v in vehicles_beta:
    for i in points_start + points_holding_1:
        departure[(v, i)] = model.NewIntVar(0, 10000, f'd_{v}{i}')
        arrival[(v, i)] = model.NewIntVar(0, 10000, f'a_{v}{i}')

Constraints

In [5]:
vehicle_capacity = 3   ## since each node visited once, no. of personnel at node <= min vehicle capacity
loading_unloading_time = 5

## Vehicle arrives and departs from same node if node is visited
for v in vehicles_alpha:
    for i in points_holding_2:
        arrive_at_holding = [alpha_v[var] for var in alpha_v if (var[0]==v and var[2]==i)]
        depart_from_holding = [alpha_v[var] for var in alpha_v if (var[0]==v and var[1]==i)]
        model.Add(sum(arrive_at_holding) == node_v[(v, i)])
        model.Add(sum(depart_from_holding) == node_v[(v, i)])
    # for i in points_destination:
    #     arrive_at_destination = [alpha_v[var] for var in alpha_v if (var[0]==v and var[2]==i)]
    #     depart_from_destination = [alpha_v[var] for var in alpha_v if (var[0]==v and var[1]==i)]
    #     model.Add(sum(arrive_at_destination) == node_v[(v, i)])
    #     model.Add(sum(depart_from_destination) == node_v[(v, i)])
for v in vehicles_beta:
    for i in points_holding_1:
        arrive_at_holding = [beta_v[var] for var in beta_v if (var[0]==v and var[2]==i)]
        depart_from_holding = [beta_v[var] for var in beta_v if (var[0]==v and var[1]==i)]
        model.Add(sum(arrive_at_holding) == node_v[(v, i)])
        model.Add(sum(depart_from_holding) == node_v[(v, i)])
    # for i in set(points_start) - set(points_launch):
    #     arrive_at_start = [beta_v[var] for var in beta_v if (var[0]==v and var[2]==i)]
    #     depart_from_start = [beta_v[var] for var in beta_v if (var[0]==v and var[1]==i)]
    #     model.Add(sum(arrive_at_start) == node_v[(v, i)])
    #     model.Add(sum(depart_from_start) == node_v[(v, i)])

## Personnel arrives and departs from same holding
for n in range(num_personnel): 
    for i, j in zip(points_holding_1, points_holding_2):
        arrive_at_holding = [beta_p[var] for var in beta_p if (var[0]==n and var[3]==i)]
        depart_from_holding = [alpha_p[var] for var in alpha_p if (var[0]==n and var[2]==j)]
        model.Add(sum(arrive_at_holding) == node_p[n, i])
        model.Add(sum(depart_from_holding) == node_p[n, i])

## Personnel visits node only if it boards the vehicle
for n in range(num_personnel):
    for j in points_destination:
        arrive_at_node = [var for var in alpha_p if (var[0]==n and var[3]==j)]
        total = 0
        for var in arrive_at_node:
            xp = model.NewIntVar(0, 10000, 'xp')   ## temporary variable for nonlinear constraint
            model.AddMultiplicationEquality(xp, alpha_p[var], alpha_v[var[-3:]])
            total += xp
        model.Add(node_p[n, j] == total)
for n in range(num_personnel):
    for j in points_holding_1:
        take_route = [var for var in beta_p if (var[0]==n and var[3]==j)]
        total = 0
        for var in take_route:
            yq = model.NewIntVar(0, 10000, 'yq')
            model.AddMultiplicationEquality(yq, beta_p[var], beta_v[var[-3:]])
            total += yq
        model.Add(node_p[n, j] == total)

## Personnel boards vehicle only if it is at that node
for n in range(num_personnel):
    for i in points_holding_2:
        arrive_at_destination = [var for var in alpha_p if (var[0]==n and var[2]==i)]
        total = 0
        for var in arrive_at_destination:
            xp = model.NewIntVar(0, 10000, 'xp')   ## temporary variable for nonlinear constraint
            model.AddMultiplicationEquality(xp, alpha_p[var], alpha_v[var[-3:]])
            total += xp
        model.Add(node_p[n, i] == total)
for n in range(num_personnel):
    for i in points_start:
        arrive_at_holding = [var for var in beta_p if (var[0]==n and var[2]==i)]
        total = 0
        for var in arrive_at_holding:
            yq = model.NewIntVar(0, 10000, 'yq')   ## temporary variable for nonlinear constraint
            model.AddMultiplicationEquality(yq, beta_p[var], beta_v[var[-3:]])
            total += yq
        model.Add(node_p[n, i] == total)

## No personnel going backwards in direction
for i in points_destination:
    depart_from_destination = [alpha_p[var] for var in alpha_p if var[2]==i]
    model.Add(sum(depart_from_destination) == 0)
for i in points_holding_1:
    depart_from_holding = [alpha_p[var] for var in alpha_p if var[2]==i]
    model.Add(sum(depart_from_holding) == 0)

## All personnel must end up at destination
for n in range(num_personnel):
    arrive_at_destination = [node_p[var] for var in node_p if ((var[0]==n) and (var[1] in points_destination))]
    model.Add(sum(arrive_at_destination) == 1)

# Vehicle capacity constraints
# for i in points_start:
#     for j in points_holding_1:
#         arrive_at_holding = [var for var in beta_p if (var[2]==i and var[3]==j)]
#         total = 0
#         for var in arrive_at_holding:
#             yq = model.NewIntVar(0, 10000, 'yq')   ## temporary variable for nonlinear constraint
#             model.AddMultiplicationEquality(yq, beta_p[var], beta_v[var[2:]])
#             total += yq
#         model.Add(total <= vehicle_capacity)
# for i in points_holding_2:
#     for j in points_destination:
#         arrive_at_destination = [var for var in alpha_p if (var[2]==i and var[3]==j)]
#         total = 0
#         for var in arrive_at_destination:
#             xp = model.NewIntVar(0, 10000, 'xp')   ## temporary variable for nonlinear constraint
#             model.AddMultiplicationEquality(xp, alpha_p[var], alpha_v[var[2:]])
#             total += xp
#         model.Add(total <= vehicle_capacity)

## Departure time after arrival time
for v_a in vehicles_alpha:
    for i in points_holding_2:
        model.Add(departure[(v_a, i)] >= arrival[(v_a, i)])
        for v_b in vehicles_beta:
            model.Add(departure[(v_a, i)] >= arrival[(v_b, points_holding_dict[i])])
    for i in points_destination:
        model.Add(departure[(v_a, i)] >= arrival[(v_a, i)])   ## we don't add in loading_unloading_time coz each vehicle does not visit all nodse
for v in vehicles_beta:
    for i in points_holding_1:
        model.Add(departure[(v, i)] >= arrival[(v, i)])
    for i in set(points_start) - set(points_launch):
        model.Add(departure[(v, i)] >= arrival[(v, i)])


## Arrival time at node = departure time + travel time
for v in vehicles_alpha:
    for j in points_destination:
        arrive_at_node = [var for var in alpha_v if (var[0]==v and var[2]==j)]
        total = 0
        for var in arrive_at_node:
            pd = model.NewIntVar(0, 10000, 'pd')   ## temporary variable for nonlinear constraint
            model.AddMultiplicationEquality(pd, alpha_v[var], departure[var[:2]])
            total += pd + alpha_v[var] * routes.loc[var[1:]]
        model.Add(arrival[(v, j)] == total)
    for j in points_holding_2:  ## alpha
        arrive_at_node = [var for var in alpha_v if (var[0]==v and var[2]==j)]
        total = 0
        for var in arrive_at_node:
            qd = model.NewIntVar(0, 10000, 'qd')
            model.AddMultiplicationEquality(qd, alpha_v[var], departure[var[:2]])
            total += qd + alpha_v[var] * routes.loc[var[1:]]
        model.Add(arrival[(v, j)] == total)
for v in vehicles_beta:
    for j in points_holding_1:  ## beta
        arrive_at_node = [var for var in beta_v if (var[0]==v and var[2]==j)]
        total = 0
        for var in arrive_at_node:
            qd = model.NewIntVar(0, 10000, 'qd')
            model.AddMultiplicationEquality(qd, beta_v[var], departure[var[:2]])
            total += qd + beta_v[var] * routes.loc[var[1:]]
        model.Add(arrival[(v, j)] == total)
    for j in points_start:
        arrive_at_node = [var for var in beta_v if (var[0]==v and var[2]==j)]
        total = 0
        for var in arrive_at_node:
            qd = model.NewIntVar(0, 10000, 'qd')
            model.AddMultiplicationEquality(qd, beta_v[var], departure[var[:2]])
            total += qd + beta_v[var] * routes.loc[var[1:]]
        model.Add(arrival[(v, j)] == total)

# ## Subtour elimination constraints
# # for q in beta_v:
# #     model.Add(arrival[q[1]] >= arrival[q[0]] - 100*(1-beta_v[q]))
# # for q in alpha_v:
# #     model.Add(arrival[q[1]] >= arrival[q[0]] - 100*(1-alpha_v[q]))

# ## No subtour (btw 2 nodes only)
# for i in points_start:
#     for j in points_holding_1:
#         model.Add(beta_v[i, j] + beta_v[j, i] <= 1)
# for i in points_holding_2:
#     for j in points_destination:
#         model.Add(alpha_v[i, j] + alpha_v[j, i] <= 1)

## All nodes are visited once only by any vehicle (MAKE SURE sufficient no. of start/holding/destination)
for i in points:
    visited_node = [node_v[var] for var in node_v if var[1]==i]
    model.Add(sum(visited_node) <= 1)

## Launch-off points can only be exited once by one vehicle
for i in points_launch:
    depart_from_start = [beta_v[var] for var in beta_v if var[1]==i]
    model.Add(sum(depart_from_start) == 1)

## If personnel at beta holding, personnel will be at alpha holding
for n in range(num_personnel):
    for h_b, h_a in zip(points_holding_1, points_holding_2):
        model.Add(node_p[n, h_b] == node_p[n, h_a])

Initial state

In [6]:
# model.Add(alpha_v[('B', 'D')] == 1)
# model.Add(beta_p[(1, 0, 'A', 'C')] == 1)

model.Add(departure['B1', 'S1'] == 0)
model.Add(departure['B2', 'S2'] == 0)
model.Add(beta_v['B1', 'S1', 'Hb1'] == 1)
model.Add(beta_v['B2', 'S3', 'Hb2'] == 1)
model.Add(node_p[(0, 'S1')] == 1)
model.Add(node_p[(1, 'S1')] == 1)
model.Add(node_p[(2, 'S3')] == 1)
model.Add(node_p[(3, 'S3')] == 1)
model.Add(node_p[(4, 'S3')] == 1)

Objective function

In [7]:
model.Minimize(sum([arrival[(v, i)] for i in points_destination for v in vehicles_alpha])) # since departure is 0, just minimise max arrival time?
# model.Minimize(max(list(arrival.values())) - max(list(departure.values()))) # cannot find max/min of decision variables
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('objective value:', solver.ObjectiveValue())
elif status == cp_model.INFEASIBLE:
    print("No solution found")
else:
    print("Something is wrong, check the status and the log of the solve")

objective value: 34.0


Results

In [8]:
# print("\nnode_v")
# for k, v in node_v.items():
#     print(k, solver.Value(v))

print("\nnode_p")
test = pandas.DataFrame(index=range(num_personnel), columns=points)
for i in node_p:
    test.loc[i[0], i[1]] = solver.Value(node_p[i])
print(test)

print("\narrival & departure")
for k, v in zip(arrival.items(), departure.items()):
    print(k[0], solver.Value(k[1]), solver.Value(v[1]))

print("\nalpha_v")
for k, v in alpha_v.items():
    print(k, solver.Value(v))

print("\nbeta_v")
for k, v in beta_v.items():
    print(k, solver.Value(v))

print("\nalpha_p")
for k, v in alpha_p.items():
    print(k, solver.Value(v))

print("\nbeta_p")
for k, v in beta_p.items():
    print(k, solver.Value(v))


node_p
  S1 S2 S3 Hb1 Hb2 Ha1 Ha2 D1 D2 D3
0  1  0  0   1   0   1   0  1  0  0
1  1  0  0   1   0   1   0  1  0  0
2  0  0  1   0   1   0   1  0  0  1
3  0  0  1   0   1   0   1  0  0  1
4  0  0  1   0   1   0   1  0  0  1

arrival & departure
('A1', 'Ha1') 8 8
('A1', 'Ha2') 11 11
('A1', 'D1') 15 15
('A1', 'D2') 0 0
('A1', 'D3') 19 19
('B1', 'S1') 0 0
('B1', 'S2') 13 13
('B1', 'S3') 0 0
('B1', 'Hb1') 8 8
('B1', 'Hb2') 0 0
('B2', 'S1') 0 0
('B2', 'S2') 0 0
('B2', 'S3') 10 0
('B2', 'Hb1') 0 0
('B2', 'Hb2') 5 5

alpha_v
('A1', 'Ha1', 'D1') 1
('A1', 'D1', 'Ha1') 0
('A1', 'Ha1', 'D2') 0
('A1', 'D2', 'Ha1') 1
('A1', 'Ha1', 'D3') 0
('A1', 'D3', 'Ha1') 0
('A1', 'Ha2', 'D1') 0
('A1', 'D1', 'Ha2') 0
('A1', 'Ha2', 'D2') 0
('A1', 'D2', 'Ha2') 1
('A1', 'Ha2', 'D3') 1
('A1', 'D3', 'Ha2') 0

beta_v
('B1', 'S1', 'Hb1') 1
('B1', 'Hb1', 'S1') 0
('B1', 'S1', 'Hb2') 0
('B1', 'Hb2', 'S1') 0
('B1', 'S2', 'Hb1') 0
('B1', 'Hb1', 'S2') 1
('B1', 'S2', 'Hb2') 0
('B1', 'Hb2', 'S2') 0
('B1', 'S3', 'Hb1') 0
('B1',

Visualise

In [9]:
print("\nbeta_v")

beta_v_list = [k for k, v in beta_v.items() if solver.Value(v)]
beta_v_dict = {k[0]: k for k in beta_v_list}
beta_v_route = []
beta_v_start = 'A'
while True:
    try:
        sublist = beta_v_dict[beta_v_start]
    except KeyError:
        break
    beta_v_route.append(sublist)
    del beta_v_dict[beta_v_start]
    beta_v_start = sublist[-1]

beta_v_route


beta_v


[]

In [10]:
print("\nalpha_v")

alpha_v_list = [k for k, v in alpha_v.items() if solver.Value(v)]
alpha_v_dict = {k[1]: k for k in alpha_v_list}
alpha_v_route = []
alpha_v_end = 'F'
while True:
    try:
        sublist = alpha_v_dict[alpha_v_end]
    except KeyError:
        break
    alpha_v_route.insert(0, sublist)
    del alpha_v_dict[alpha_v_end]
    alpha_v_end = sublist[0]

alpha_v_route


alpha_v


[]